In [87]:
import pandas as pd
import re

In [88]:
# Read the preprocessed data
df = pd.read_csv('a:/df_cleaned.csv')
df.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street..."
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob..."
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ..."
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'..."
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee..."


In [89]:
# Define your regex pattern
pattern = re.compile(r"\b(bloed(braken)?|hematemesis|melena|maagdarmbloeding|gastro[- ]intestinale bloeding|rectaal bloedverlies|vers bloed per anum|bloeding uit het maag[- ]darmkanaal)\b", re.IGNORECASE)

# Apply regex
df['agib_mention'] = df['alltext'].apply(lambda x: bool(pattern.search(str(x))))

# Optionally exclude negated matches
negation_pattern = re.compile(r"\b(geen|niet|zonder)\b.{0,20}?\b(bloed(braken)?|hematemesis|melena|maagdarmbloeding|gastro[- ]intestinale bloeding|rectaal bloedverlies|vers bloed per anum|bloeding uit het maag[- ]darmkanaal)\b", re.IGNORECASE)
df['negated'] = df['alltext'].apply(lambda x: bool(negation_pattern.search(str(x))))
df['agib_final'] = df['agib_mention'] & ~df['negated']

In [90]:
# df_agib = df[df['agib_final'] == True]
# df_agib

In [91]:
print(df_agib['verslagen_report_content'].iloc[0])

Dhr. A.J. Dingemans, huisarts
[STREETNAME] NR  [CITY]





datum
29-11-2020
kenmerk
0007475372 / [PATIENTID]
BSN nr.
[BSN]
betreft
mevrouw [INITIALS] [LASTNAME], geb. [BIRTHDATE] (87)
[STREETNAME], [ZIP]  [CITY]
tel. [PHONENUMBER]

 
Geachte collega,

Bovengenoemde patiënte was opgenomen van 24-11-2020 tot en met 26-11-2020 op de afdeling Maag-, Darm- en Leverziekten in verband met melaena en rectaal bloedverlies. 

Voorgeschiedenis
2002 Diep veneuze trombose en longembolie
2013 Cholecystectomie
2015 Diverticulitis
2016 Atriumfibrilleren met spontane conversie naar sinusritme
2016 Melena, waarvoor geen verklaring werd gevonden. In verband met stabiel Hb en in overleg met patiënte expectatief beleid. 
2018 Vermoeidheid bij sinusbradycardie waarvoor stop metoprolol en tambocor. 

Anamnese
Vanmiddag rond 13.30u fors helderrood bloedverlies met stolsels. Vermengd met ontlasting, mogelijk was deze zwart van kleur. Sinds 5 dagen zeurende pijn in de bovenbuik; maagpijn, waarvoor ze is gestopt

In [92]:
# Define AGIB regex pattern
agib_pattern = re.compile(
    r"\b(bloedbraken|bloed braken|hematemesis|melena|maagdarmbloeding|gastro[- ]intestinale bloeding|rectaal bloedverlies|vers bloed per anum|bloeding uit het maag[- ]darmkanaal)\b",
    re.IGNORECASE
)

# Function to extract match with 5 words before and after
def extract_context(text, pattern, window=5):
    matches = []
    words = re.findall(r'\w+|\W', text)  # keep punctuation separate
    for match in pattern.finditer(text):
        # Find position in word list
        match_text = match.group()
        start, end = match.start(), match.end()

        # Convert text index to word index
        char_idx = 0
        for i, word in enumerate(words):
            char_idx += len(word)
            if char_idx >= start:
                match_index = i
                break

        # Extract window
        start_idx = max(0, match_index - window)
        end_idx = min(len(words), match_index + window + 1)
        context = ''.join(words[start_idx:end_idx])
        matches.append(context.strip())

    return matches[0] if matches else None

# Apply function
df['agib_context'] = df['alltext'].apply(lambda x: extract_context(str(x), agib_pattern))
df.head()
# Function to extract the exact match
def extract_match(text, pattern):
    match = pattern.search(text)
    return match.group() if match else None

# Apply function to extract the exact match
df['agib_match'] = df['alltext'].apply(lambda x: extract_match(str(x), agib_pattern))
df.head()


,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_mention,negated,agib_final,agib_context,agib_match
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",True,False,True,spontaan conversie sinusritme melena waarvoor ...,melena
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob...",True,False,True,None,None
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ...",True,False,True,kamphuis indicatie bloedverliesrectaal melena ...,melena
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'...",True,False,True,jt kamphuis indicatie Melena bloedverlie per,Melena
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee...",True,False,True,beloop vpk gisteravond melena vannacht donkerrood,melena


In [93]:
df

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_mention,negated,agib_final,agib_context,agib_match
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",True,False,True,spontaan conversie sinusritme melena waarvoor ...,melena
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob...",True,False,True,None,None
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ...",True,False,True,kamphuis indicatie bloedverliesrectaal melena ...,melena
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'...",True,False,True,jt kamphuis indicatie Melena bloedverlie per,Melena
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee...",True,False,True,beloop vpk gisteravond melena vannacht donkerrood,melena
...,...,...,...,...,...,...,...,...,...,...,...,...
9572,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Consult,Samenvatting: \n1e consult\r\n-Type 1e consult...,2015-03-20 08:13:00,2015-03-20 08:13:00,samenvatting consult type consult uitbreiden a...,"['samenvatting', 'consult', 'type', 'consult',...",False,False,False,None,None
9573,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, Kliniek: vervolgconsult",Samenvatting: \nDecursus\r\n-Type decursus: De...,2015-01-14 15:39:00,2015-01-14 15:39:00,samenvatting decursus type decursus decursus s...,"['samenvatting', 'decursus', 'type', 'decursus...",False,False,False,None,None
9574,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nVerpleegkundige verslaglegging...,2014-12-21 09:31:00,2014-12-21 09:31:00,samenvatting verpleegkundig verslaglegging ver...,"['samenvatting', 'verpleegkundig', 'verslagleg...",False,False,False,None,None
9575,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nMedisch Dossier\r\n[ Vk Sputov...,2010-11-10 21:03:00,2010-11-10 21:03:00,samenvatting medisch dossier vk sputovamo leef...,"['samenvatting', 'medisch', 'dossier', 'vk', '...",False,False,False,None,None


In [94]:
# Filter to rows where AGIB was found
df_agib = df[df['agib_context'].notnull()]
df_agib

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_mention,negated,agib_final,agib_context,agib_match
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",True,False,True,spontaan conversie sinusritme melena waarvoor ...,melena
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ...",True,False,True,kamphuis indicatie bloedverliesrectaal melena ...,melena
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'...",True,False,True,jt kamphuis indicatie Melena bloedverlie per,Melena
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee...",True,False,True,beloop vpk gisteravond melena vannacht donkerrood,melena
5,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Beloop: \nVerpleging belt: \nPatiënte heeft no...,2020-11-24 20:53:00,2020-11-24 20:53:00,beloop verpleging blen Patiënte fors hoeveelhe...,"['beloop', 'verpleging', 'blen', 'Patiënte', '...",True,False,True,Patiënte fors hoeveelheid melena hierbij tensie,melena
...,...,...,...,...,...,...,...,...,...,...,...,...
9164,F8C241E69061BB450778B7A4CC336FF520469A0A,"Consult, SEH",Reden van komst / Verwijzing: \nReden van koms...,2020-05-31 15:02:00,2020-05-31 15:02:00,reden komst verwijzing reden komst brief recta...,"['reden', 'komst', 'verwijzing', 'reden', 'kom...",True,False,True,normaal consistentie kleur melena heden buikkl...,melena
9262,F8D116F01EE0039678998F393FA337C10AD4F4E2,Klinische Brief,Aan de weledelgeleerde vrouwe\r\ndrs. H.N.J. G...,2020-03-25 10:50:00,2020-03-25 10:50:00,weledelgeleer vrouwe drs hnj gie huisarts stre...,"['weledelgeleer', 'vrouwe', 'drs', 'hnj', 'gie...",True,False,True,donker kleur riekken melena bloed aanvullen,melena
9264,F8D116F01EE0039678998F393FA337C10AD4F4E2,Spoedeisende Hulp,Aan de weledelgeleerde vrouwe\r\ndrs. H.N.J. G...,2020-03-24 12:29:00,2020-03-24 12:29:00,weledelgeleer vrouwe drs hnj gie huisarts stre...,"['weledelgeleer', 'vrouwe', 'drs', 'hnj', 'gie...",True,False,True,donker kleur riekken melena bloed aanvullen,melena
9266,F8D116F01EE0039678998F393FA337C10AD4F4E2,"Consult, SEH",Reden van komst / Verwijzing: \nReden van koms...,2020-03-24 10:00:00,2020-03-24 10:00:00,reden komst verwijzing reden komst brief dyspn...,"['reden', 'komst', 'verwijzing', 'reden', 'kom...",True,False,True,donker kleur riekken melena bloed aanvullen,melena


In [95]:
df_agib['agib_context'].iloc[4]

'Patiënte fors hoeveelheid melena hierbij tensie'

In [96]:
pseudo_ids_with_agib = df[df['agib_mention'] == True]['pseudo_id'].unique()
pseudo_ids_df = pd.DataFrame(pseudo_ids_with_agib, columns=['pseudo_id'])
pseudo_ids_df

,pseudo_id
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6
1,088C9FD98B8B2CBCB597C17C07AC1845B21F0849
2,0A5645E02FA818D1629926B6BEFA81CF91C25A46
3,0BC512A65442D0BB4B00FBE05E7EA6283E5C11FD
4,0DD73490997F61870C32B3BB7C9CDE4E801FFF7D
...,...
100,F797097BA39F81B05EEC3F251973743CA03CB9B4
101,F8C241E69061BB450778B7A4CC336FF520469A0A
102,F8D116F01EE0039678998F393FA337C10AD4F4E2
103,FAA79717FF2C725767E9469350ACECF640E5FCBC


In [97]:
# Read the preprocessed data
df_specific = pd.read_csv('a:/df_cleaned.csv')
df_specific.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street..."
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob..."
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ..."
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'..."
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee..."


In [98]:
# Define your regex pattern
pattern = re.compile(r"\b(maagdarmbloeding|gastro[- ]intestinale bloeding|bloeding uit het maag[- ]darmkanaal|gib|bloedverlies gastro[- ]intestinaal)\b", re.IGNORECASE)

# Apply regex
df_specific['agib_mention'] = df_specific['alltext'].apply(lambda x: bool(pattern.search(str(x))))

# Optionally exclude negated matches
negation_pattern = re.compile(r"\b(geen|niet|zonder)\b.{0,20}?\b(bloed(braken)?|hematemesis|melena|maagdarmbloeding|gastro[- ]intestinale bloeding|rectaal bloedverlies|vers bloed per anum|bloeding uit het maag[- ]darmkanaal)\b", re.IGNORECASE)
df_specific['negated'] = df_specific['alltext'].apply(lambda x: bool(negation_pattern.search(str(x))))
df_specific['agib_final'] = df_specific['agib_mention'] & ~df['negated']

In [99]:
# Define AGIB regex pattern
agib_pattern = re.compile(
    r"\b(maagdarmbloeding|gastro[- ]intestinale bloeding|bloeding uit het maag[- ]darmkanaal|gib|bloedverlies gastro[- ]intestinaal)\b", 
    re.IGNORECASE
    )

# Function to extract match with 5 words before and after
def extract_context(text, pattern, window=5):
    matches = []
    words = re.findall(r'\w+|\W', text)  # keep punctuation separate
    for match in pattern.finditer(text):
        # Find position in word list
        match_text = match.group()
        start, end = match.start(), match.end()

        # Convert text index to word index
        char_idx = 0
        for i, word in enumerate(words):
            char_idx += len(word)
            if char_idx >= start:
                match_index = i
                break

        # Extract window
        start_idx = max(0, match_index - window)
        end_idx = min(len(words), match_index + window + 1)
        context = ''.join(words[start_idx:end_idx])
        matches.append(context.strip())

    return matches[0] if matches else None

# Apply function
df_specific['agib_context'] = df_specific['alltext'].apply(lambda x: extract_context(str(x), agib_pattern))
df_specific.head()

# Apply function to extract the exact match
df_specific['agib_match'] = df_specific['alltext'].apply(lambda x: extract_match(str(x), agib_pattern))
df_specific.head()


,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_mention,negated,agib_final,agib_context,agib_match
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",False,False,False,None,None
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob...",False,False,False,None,None
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ...",False,False,False,None,None
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'...",False,False,False,None,None
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee...",False,False,False,None,None


In [100]:
# Filter to rows where AGIB was found
df_specific_agib = df_specific[df_specific['agib_context'].notnull()]
df_specific_agib

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_mention,negated,agib_final,agib_context,agib_match
5900,A9A2482C557733BFC15A7DEAC217F8766D37FAA1,"Consult, Polikliniek: eerste consult",Samenvatting: \nEerste consult\r\n-Type 1e con...,2011-08-17 15:26:00,2011-08-17 15:26:00,samenvatting consult type consult uitbreiden a...,"['samenvatting', 'consult', 'type', 'consult',...",True,False,True,samenvatting conclusie st maagdarmbloeding dif...,maagdarmbloeding


In [101]:
pseudo_ids_with_agib = df_specific[df_specific['agib_mention'] == True]['pseudo_id'].unique()
pseudo_ids_df_specific = pd.DataFrame(pseudo_ids_with_agib, columns=['pseudo_id'])
pseudo_ids_df_specific

,pseudo_id
0,A9A2482C557733BFC15A7DEAC217F8766D37FAA1


In [103]:
df.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_mention,negated,agib_final,agib_context,agib_match
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street...",True,False,True,spontaan conversie sinusritme melena waarvoor ...,melena
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob...",True,False,True,None,None
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ...",True,False,True,kamphuis indicatie bloedverliesrectaal melena ...,melena
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'...",True,False,True,jt kamphuis indicatie Melena bloedverlie per,Melena
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee...",True,False,True,beloop vpk gisteravond melena vannacht donkerrood,melena


In [106]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
dutch_stopwords = stopwords.words('dutch')

# Filter known positive reports
positive_reports = df[df['agib_final'] == True]['alltext'].dropna()

# Create word frequency matrix
vectorizer = CountVectorizer(lowercase=True, stop_words=dutch_stopwords, max_features=1000)
X = vectorizer.fit_transform(positive_reports)

# Get top terms
word_counts = X.sum(axis=0).A1
vocab = vectorizer.get_feature_names_out()
freq_df = pd.DataFrame({'term': vocab, 'count': word_counts}).sort_values(by='count', ascending=False)

print(freq_df.head(30))


             term  count
555            mg   6848
217           dag   6227
689           per   5564
652         oraal   4102
830          stuk   3841
571         mmoll   3369
840        tablet   3252
105        beleid   3180
639     onderzoek   3162
55         anemie   2761
367            hb   2634
681       patiënt   2622
649        opname   2379
452           ivm   2360
223            dd   2330
123         bloed   2169
131   bloedverlie   2077
53       anamnees   2007
610       normaal   1936
354          goed   1907
990          zien   1904
194     conclusie   1884
771  samenvatting   1812
201      controle   1808
890            ul   1717
641    ontlasting   1678
547        melena   1623
966      waarvoor   1608
5       aanvullen   1465
258            dr   1444


In [107]:
# Filter known positive reports
positive_reports = df[df['agib_final'] == False]['alltext'].dropna()

# Create word frequency matrix
vectorizer = CountVectorizer(lowercase=True, stop_words=dutch_stopwords, max_features=1000)
X = vectorizer.fit_transform(positive_reports)

# Get top terms
word_counts = X.sum(axis=0).A1
vocab = vectorizer.get_feature_names_out()
freq_df = pd.DataFrame({'term': vocab, 'count': word_counts}).sort_values(by='count', ascending=False)

print(freq_df.head(30))

                 term  count
546                mg   8282
754      samenvatting   7978
96             beleid   7522
202               dag   5989
666               per   5764
627         onderzoek   4994
208                dd   4831
439               ivm   4752
47             anemie   4713
181         conclusie   4332
635          opdracht   4302
686              poli   4201
641             oraal   4060
344              goed   3868
187          controle   3820
724            rechts   3713
810              stuk   3637
639            opname   3577
968          waarvoor   3550
44           anamnees   3512
26           afspraak   3494
95             bekend   3383
472               lab   3325
956  voorgeschiedenis   3305
662           patiënt   3259
392       hypertensie   3243
354                hb   3217
820            tablet   3016
988              zien   3002
562             mmoll   2983


In [111]:
print(df[df['agib_final'] == True])#['alltext'].dropna()

                                     pseudo_id  \
0     046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6   
1     046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6   
2     046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6   
3     046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6   
4     046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6   
...                                        ...   
9526  FD8C682C1F4FDA1E5EC0B760D30875556419BD71   
9527  FD8C682C1F4FDA1E5EC0B760D30875556419BD71   
9529  FD8C682C1F4FDA1E5EC0B760D30875556419BD71   
9531  FD8C682C1F4FDA1E5EC0B760D30875556419BD71   
9534  FD8C682C1F4FDA1E5EC0B760D30875556419BD71   

                 verslagen_report_tags  \
0                      Klinische Brief   
1     Consult, Kliniek: vervolgconsult   
2                  Poliklinische Brief   
3                  Poliklinische Brief   
4     Consult, Kliniek: vervolgconsult   
...                                ...   
9526                           Consult   
9527                           Consult   
9529  Consult, Klinie

In [110]:
df[df['agib_final'] == False]#['alltext'].dropna()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens,agib_mention,negated,agib_final,agib_context,agib_match
18,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, SEH",Samenvatting: \nVerpleegkundige verslaglegging...,2016-08-22 15:14:00,2016-08-22 15:14:00,samenvatting verpleegkundig verslaglegging ver...,"['samenvatting', 'verpleegkundig', 'verslagleg...",False,False,False,None,None
19,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, SEH",Samenvatting: \nApotheek\r\n-Apotheek akkoord: ja,2016-06-07 12:14:00,2016-06-07 12:14:00,samenvatting apotheek apotheek akkoord,"['samenvatting', 'apotheek', 'apotheek', 'akko...",False,False,False,None,None
20,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nVervolgconsult\r\n-Type regist...,2015-07-16 09:58:00,2015-07-16 09:58:00,samenvatting vervolgconsult type registratie u...,"['samenvatting', 'vervolgconsult', 'type', 're...",False,False,False,None,None
21,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, SEH",Samenvatting: \nMedisch Dossier\r\n[ Vk Sputov...,2015-06-03 18:25:00,2015-06-03 18:25:00,samenvatting medisch dossier vk sputovamo leef...,"['samenvatting', 'medisch', 'dossier', 'vk', '...",False,False,False,None,None
22,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, SEH",Samenvatting: \nVerpleegkundige verslaglegging...,2015-06-03 17:40:00,2015-06-03 17:40:00,samenvatting verpleegkundig verslaglegging ver...,"['samenvatting', 'verpleegkundig', 'verslagleg...",False,False,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
9572,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Consult,Samenvatting: \n1e consult\r\n-Type 1e consult...,2015-03-20 08:13:00,2015-03-20 08:13:00,samenvatting consult type consult uitbreiden a...,"['samenvatting', 'consult', 'type', 'consult',...",False,False,False,None,None
9573,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, Kliniek: vervolgconsult",Samenvatting: \nDecursus\r\n-Type decursus: De...,2015-01-14 15:39:00,2015-01-14 15:39:00,samenvatting decursus type decursus decursus s...,"['samenvatting', 'decursus', 'type', 'decursus...",False,False,False,None,None
9574,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nVerpleegkundige verslaglegging...,2014-12-21 09:31:00,2014-12-21 09:31:00,samenvatting verpleegkundig verslaglegging ver...,"['samenvatting', 'verpleegkundig', 'verslagleg...",False,False,False,None,None
9575,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nMedisch Dossier\r\n[ Vk Sputov...,2010-11-10 21:03:00,2010-11-10 21:03:00,samenvatting medisch dossier vk sputovamo leef...,"['samenvatting', 'medisch', 'dossier', 'vk', '...",False,False,False,None,None
